## 6.1.4 Generate Random Latitudes and Longitudes

In [1]:
from citipy import citipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

In [3]:
# add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# create a list for holding cities
cities = []

# identify the nearest city for each latitude and longitude combination
for c in coordinates: 
    lat, lng = c[0], c[1]
    city = citipy.nearest_city(lat, lng).city_name
    
    if city not in cities:
        # if the name is unique, then add to the cities list
        cities.append(city)

# print the city count to confirm sufficient count
len(cities)

633